In [49]:
#Dependencies
import pandas as pd 
import numpy as np 
import gmaps
import json

from gconfig import gkey
gmaps.configure(api_key=gkey)

In [21]:
file_path = "Results/WeatherPy/citydata.csv"
citydata_df = pd.read_csv(file_path)
citydata_df.head()

,Unnamed: 0,Cities,Lat,Lon,Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,0,new norfolk,-42.78,147.06,52.20,64,67,4.00
1,1,tuktoyaktuk,69.45,-133.04,37.40,71,90,8.05
2,2,punta arenas,-53.15,-70.92,41.00,75,20,10.29
3,3,hanting,36.77,119.21,77.45,67,59,2.35
4,4,rikitea,-23.12,-134.97,72.28,80,21,12.08


In [16]:
locations = citydata_df[["Lat","Lon"]]
humidity = citydata_df["Humidity (%)"].astype(float)

In [33]:
figure_layout = {"width":"400px","height":"600px","margin":"0 auto 0 auto"}
fig = gmaps.figure(figure_layout,center=(46.0,-5.0),zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=100,point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [48]:
temp_df = citydata_df.loc[citydata_df["Temp (F)"] > 70]
good_temp_df = temp_df.loc[temp_df["Temp (F)"] <= 80]
no_wind_df = good_temp_df.loc[good_temp_df["Wind Speed (mph)"] < 10]
no_clouds_df = no_wind_df.loc[no_wind_df["Cloudiness (%)"] == 0]
dest_df = no_clouds_df.loc[no_clouds_df["Humidity (%)"] < 75]
dest_df.head(9)

,Unnamed: 0,Cities,Lat,Lon,Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
40,45,manono,-7.30,27.42,72.70,30,0,4.05
249,280,korla,41.76,86.15,71.01,52,0,1.99
293,329,silopi,37.25,42.47,79.29,18,0,2.01
356,401,mudu,31.25,120.51,78.01,69,0,5.66
363,408,alice springs,-23.70,133.88,71.89,18,0,9.17
378,425,najran,17.49,44.13,77.00,27,0,2.35
412,468,palagiano,40.58,17.04,72.37,65,0,1.95
458,518,hun,29.13,15.95,74.79,56,0,6.64
544,620,darab,28.75,54.54,74.35,37,0,1.43


In [77]:
#target_coords = dest_df[["Lat","Lon"]]
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

#test result
params = {"location" : "41.76,86.15", "keyword" : target_search, "radius" : target_radius, "type" : target_type, "key" : gkey}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

response = requests.get(base_url, params=params).json()
print(json.dumps(response,indent=4,sort_keys=True))

                    "lng": 86.15018707010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png",
            "name": "Haiyu Love Theme Hotel",
            "place_id": "ChIJBVlJG2nREzgRvdU6aaolP18",
            "plus_code": {
                "compound_code": "Q572+7M Tianshan Residential District, Korla, Bayingolin Mongol Autonomous Prefecture, Xinjiang, China",
                "global_code": "8MH8Q572+7M"
            },
            "rating": 3,
            "reference": "ChIJBVlJG2nREzgRvdU6aaolP18",
            "scope": "GOOGLE",
            "types": [
                "point_of_interest",
                "establishment"
            ],
            "user_ratings_total": 1,
            "vicinity": "Near Renmin East Road, Korla"
        },
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 41.777548,
    

In [96]:
#Add columns for hotel info
dest_df["Hotel Name"] = ""
dest_df["City"] = ""
dest_df["Country"] = ""
dest_df["Hotel Lat"] = ""
dest_df["Hotel Lon"] = ""

dest_df.head()

,Unnamed: 0,Cities,Lat,Lon,Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name,City,Country,Hotel Lat,Hotel Lon
40,45,manono,-7.30,27.42,72.70,30,0,4.05,,,,,
249,280,korla,41.76,86.15,71.01,52,0,1.99,,,,,
293,329,silopi,37.25,42.47,79.29,18,0,2.01,,,,,
356,401,mudu,31.25,120.51,78.01,69,0,5.66,,,,,
363,408,alice springs,-23.70,133.88,71.89,18,0,9.17,,,,,


In [97]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"keyword" : target_search, "radius" : target_radius, "type" : target_type, "key" : gkey}

markers_lat = []
markers_lon = []
for index, row in dest_df.iterrows():
    dest_city = row["Cities"]
    dest_lat = row["Lat"]
    dest_lon = row["Lon"]

    params["location"] = (f"{dest_lat}, {dest_lon}")

    print(f"Retrieving results for Index {index}: {dest_city}...")
    response = requests.get(base_url,params=params).json()
    results = response["results"]

    try:
        print(f"Closest hotel is {results[0]['name']}.")

        dest_df.loc[index, "Hotel Name"] = results[0]["name"]
        dest_df.loc[index, "City"] = results[0]["vicinity"]

        dest_df.loc[index, "Hotel Lat"] = results[0]["geometry"]["location"]["lat"]
        dest_df.loc[index, "Hotel Lon"] = results[0]["geometry"]["location"]["lng"]

        markers_lat.append(hotel_lat)
        markers_lon.append(hotel_lon)

    except (KeyError,IndexError):
        print(f"Missing field/result... skipping...")

    print("------------")



Retrieving results for Index 40: manono...
Closest hotel is Hotel hewa bora.
------------
Retrieving results for Index 249: korla...
Closest hotel is Kangcheng Jian Guo International Hotel.
------------
Retrieving results for Index 293: silopi...
Closest hotel is Grand Hotel.
------------
Retrieving results for Index 356: mudu...
Closest hotel is Shangri-La Hotel, Suzhou.
------------
Retrieving results for Index 363: alice springs...
Closest hotel is DoubleTree by Hilton Hotel Alice Springs.
------------
Retrieving results for Index 378: najran...
Missing field/result... skipping...
------------
Retrieving results for Index 412: palagiano...
Closest hotel is Accord Le Rose Hotel Taranto Palagiano Hotel.
------------
Retrieving results for Index 458: hun...
Closest hotel is Alrwasi Hotel.
------------
Retrieving results for Index 544: darab...
Closest hotel is Naghshe shapoor Hotel.
------------


In [98]:
dest_df.head(10)

,Unnamed: 0,Cities,Lat,Lon,Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name,City,Country,Hotel Lat,Hotel Lon
40,45,manono,-7.30,27.42,72.70,30,0,4.05,Hotel hewa bora,Manono,,-7.29285,27.3958
249,280,korla,41.76,86.15,71.01,52,0,1.99,Kangcheng Jian Guo International Hotel,"No 618 East Jiaotong Road Korla Xinjiang, Korla",,41.7721,86.159
293,329,silopi,37.25,42.47,79.29,18,0,2.01,Grand Hotel,"Şehit Harun Boy, İpek Yolu Cad., Silopi",,37.2409,42.4793
356,401,mudu,31.25,120.51,78.01,69,0,5.66,"Shangri-La Hotel, Suzhou","168 Tayuan Rd, Shishan Mountain, Huqiu Distric...",,31.2922,120.559
363,408,alice springs,-23.70,133.88,71.89,18,0,9.17,DoubleTree by Hilton Hotel Alice Springs,"82 Barrett Dr, Alice Springs",,-23.715,133.879
378,425,najran,17.49,44.13,77.00,27,0,2.35,,,,,
412,468,palagiano,40.58,17.04,72.37,65,0,1.95,Accord Le Rose Hotel Taranto Palagiano Hotel,"Viale Chiatona 105Palagiano TarantoTA 74100, T...",,40.5765,17.0411
458,518,hun,29.13,15.95,74.79,56,0,6.64,Alrwasi Hotel,houn-aljoufra,,29.1171,15.9159
544,620,darab,28.75,54.54,74.35,37,0,1.43,Naghshe shapoor Hotel,"Darab, Daneshjoo Boulevard، Pasdaran Blv.",,28.7606,54.562


In [82]:
#print(markers_lat)

[-7.292854699999999, 41.772123, 37.2408829, 31.2921597, -23.7150157, 40.576486, 29.1171477, 28.7605701]


In [88]:
#markers = zip(markers_lat,markers_lon)

In [89]:
#print(list(markers))

[(-7.292854699999999, 27.3958247), (41.772123, 86.158971), (37.2408829, 42.47926330000001), (31.2921597, 120.5590259), (-23.7150157, 133.879153), (40.576486, 17.041128), (29.1171477, 15.9158511), (28.7605701, 54.5619665)]


In [93]:
#markers_df = pd.DataFrame(markers, columns=["hotel_lat","hotel_lon"])

In [107]:
manono = gmaps.Marker((-7.29,27.4), info_box_content="Hotel hewa bora")
korla = gmaps.Marker((41.77,86.16), info_box_content="Kangcheng Jian Guo International Hotel")
silopi = gmaps.Marker((41.77,86.16), info_box_content="Grand Hotel")
mudu = gmaps.Marker((37.24,42.48), info_box_content="Shangri-La Hotel, Suzhou")
alice_springs = gmaps.Marker((-23.72,133.88), info_box_content="DoubleTree by Hilton Hotel Alice Springs")
palagiano = gmaps.Marker((40.58,17.04), info_box_content="Accord Le Rose Hotel Taranto Palagiano Hotel")
hun = gmaps.Marker((29.12,15.92), info_box_content="Alrwasi Hotel")
darab = gmaps.Marker((28.76,54.56), info_box_content="Naghshe shapoor Hotel")

drawing = gmaps.drawing_layer(features=[manono,korla,silopi,mudu,alice_springs,palagiano,hun,darab])

fig.add_layer(drawing)

fig

Figure(layout=FigureLayout(height='420px'))